In [0]:
TODO

## Effective use of DSPy involves evaluation and iterative development

You already know a lot about DSPy at this point. If all you want is quick scripting, you likely now have the skillset to be effective. Sprinkling DSPy signatures and modules into your Python control flow is a pretty ergonomic way to just get stuff done with LMs.

That said, you're likely here because you want to build high-quality systems and improve them over time. The way to do that in DSPy is to leverage an evaluation cycle along with DSPy's [Optimizers](https://dspy.ai/learn/optimization/overview/).

For our prototype we will use a bunch of StackExchange-based questions and their correct answers from the [RAG-QA Arena](https://arxiv.org/abs/2407.13998) dataset.  The dataset is prepare and divided into a train, validation, and dev sub-sets in the notebook executed below 

In [0]:
# TODO: prep data

## Evaluation in DSPy

What kind of metrics suit our question-answering task? There are many choices, but since the answers are long, we may ask: How well does the system response _cover_ all key facts in the gold response? And the other way around, how well is the system response _not saying things_ that aren't in the gold response?

The above definition is essentially a **semantic F1**, so let's load a `SemanticF1` metric from DSPy. This metric is actually implemented as a [very simple DSPy module](https://github.com/stanfordnlp/dspy/blob/77c2e1cceba427c7f91edb2ed5653276fb0c6de7/dspy/evaluate/auto_evaluation.py#L21), which makes it usable with any DSPy LM.

The metric is calculated based on:
<br>
$$Semantic F1 = \frac{2 \cdot \text{Precision} \cdot \text{Recall}}{\text{Precision} + \text{Recall}}$$
$${\text{Precission}}: {\text{fraction (out of 1.0) of system response covered by the ground truth}}$$
$${\text{Recall}}: {\text{fraction (out of 1.0) of ground truth covered by the system response}}$$

In [0]:
from dspy.evaluate import SemanticF1

# Let's pick an `example` here from the data.
example = trainset[0]
# Produce a prediction from our `cot` module, using the `example` above as input.
pred = cot(**example.inputs())
metric = SemanticF1()
score = metric(example, pred) #measure_correctness(example, pred)

For evaluation, you could use the metric above in a simple loop and just average the score. But for parallelism some additional utilities, we can rely on `dspy.Evaluate`.

In [0]:
# Define an evaluator that we can re-use.
evaluate = dspy.Evaluate(
    devset=devset,
    metric=metric, 
    num_threads=32,
    display_progress=True,
    display_table=2,
)

# Evaluate the Chain-of-Thought program.
evaluate(cot)

# Step 4: DSPy Optimization and RAG

So far, we built a very simple chain-of-thought module for question answering and evaluated it on a small dataset, but can we do better?

In the rest of this guide, we will build a retrieval-augmented generation (RAG) program in DSPy for the same task. We'll see how this can boost the score substantially, then we'll use one of the DSPy Optimizers to compile our RAG program to higher-quality prompts, raising our scores even more.

Set up your system's retriever.
As far as DSPy is concerned, you can plug in any Python code for calling tools or retrievers. Here, we'll use the Datbricks Vector Search index we set up earlier to execute a Hybrid Semantic Search (using embeddings and keywords)

In [0]:
import dspy
import mlflow 
# from dspy.retrieve.databricks_rm import DatabricksRM

class RAG(dspy.Module):
    def __init__(self, num_docs=5, for_mosaic_agent=False):

        # setup mlflow tracing
        mlflow.dspy.autolog()

        # setup flag indicating if the object will be deploy as a Mosaic Agent
        self.for_mosaic_agent = for_mosaic_agent
        
        # setup the language model
        self.lm = dspy.LM("databricks/databricks-meta-llama-3-3-70b-instruct")

        # setup the predictor and signature
        self.respond = dspy.ChainOfThought("context, question -> response")

        # setup the retriever pointing to Databricks Vector Search Index
        self.retriever = DatabricksRM(
            databricks_index_name="main.luis_moros.colbertv2_text_set_index",
            k=num_docs,
            use_with_databricks_agent_framework=for_mosaic_agent
        )

    def forward(self, question):

        if self.for_mosaic_agent:
            question = question[-1]["content"]

        context = self.retriever(
            question, 
            query_type="hybrid"# Using hybrid search (embeddigns + keywords search)
        )

        with dspy.context(lm=self.lm):
            response = self.respond(context=context, question=question)

        if self.for_mosaic_agent:
            return response.response
        return response

In [0]:
rag = RAG()
rag(question="what are high memory and low memory on linux?")

In [0]:
evaluate(rag)